# Implementation du cryptosysteme BIKE candidate pour la selection du NIST dans le cadre des systemes Post quantiques
==================================

BIKE: Bit Flipping Key Encapsulation est un systeme de cryptographique asymetrique
Son utilité reside dans la transmission securisée des clés de chiffrement pour des systemes crytographiques symetriques.
C'est à dire que BIKE permet en realité d'encapsuler (chiffrer) une clé pour un systeme cryptographique symetrique afin de transmettre celle ci à un interlocuteur.

Le syteme est decomposable en 3 principales parties:
- La generation des clés
- L'encapsulation (Chiffrement)
- La decapsulation (Déchiffrement)

## Notations à considerer dans la suite

## La generation des 

## Generation des clés
Ici is s'agit de generer h<sub>0</sub> et h<sub>1</sub> en se basant uniquement sur le niveau de securité quantique cherché l

In [ ]:
import secrets

In [ ]:
def keygen(λ):
    # λ : le niveau de sécurité quantique visé
    # on suppose que λ est valide

    # on fixe les paramètres r et w en fonction de λ
    if λ == 64:
        r = 1233
        w = 142
    elif λ == 80:
        r = 1988
        w = 198
    elif λ == 96:
        r = 2401
        w = 240
    elif λ == 128:
        r = 3277
        w = 312
    else:
        raise ValueError("λ invalide")

    # on génère h0 et h1 de poids impair w/2
    h0 = secrets.randbits(r) # un nombre aléatoire de r bits
    h1 = secrets.randbits(r) # un autre nombre aléatoire de r bits
    while bin(h0).count("1") != w//2 or h0 % 2 == 0: # tant que h0 n'a pas le bon poids ou est pair
        h0 = secrets.randbits(r) # on regénère h0
    while bin(h1).count("1") != w//2 or h1 % 2 == 0: # tant que h1 n'a pas le bon poids ou est pair
        h1 = secrets.randbits(r) # on regénère h1

    # on génère g de poids impair r/2
    g = secrets.randbits(r) # un nombre aléatoire de r bits
    while bin(g).count("1") != r//2 or g % 2 == 0: # tant que g n'a pas le bon poids ou est pair
        g = secrets.randbits(r) # on regénère g

    # on calcule f0 et f1 en multipliant g par h1 et h0
    f0 = (g * h1) % (2**r - 1) # le produit modulo X^r - 1
    f1 = (g * h0) % (2**r - 1) # le produit modulo X^r - 1

    # on renvoie la clé privée (h0, h1) et la clé publique (f0, f1)
    return (h0, h1), (f0, f1)
